In [ ]:
!pip install git+https://github.com/huggingface/diffusers
!pip install invisible_watermark transformers accelerate safetensors
!pip install bitsandbytes

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-8ufnd17o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-8ufnd17o
  Resolved https://github.com/huggingface/diffusers to commit 7a24977ce3f7b406034362c15c17b4159abe7dfd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.21.0.dev0-py3-none-any.whl size=1367521 sha256=1a92c55cb792fa17c761572e3858c4bc776e7966d411b9151919363ce5cc0c4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-3luww0wv/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:0

In [ ]:
!accelerate config default

2023-08-21 13:25:27.332660: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
!accelerate config

2023-08-21 12:17:19.731205: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)


In [ ]:
from google.colab import drive
import torch
from huggingface_hub import notebook_login
notebook_login() # login in order to be able download models

In [ ]:
want_to_convert_to_chkpt = False # if you want to convert to a checkpoint after the training
if want_to_convert_to_chkpt:
    !wget -q https://github.com/ShivamShrirao/diffusers/raw/9db565e15e8e35f1ebb48e1409b8ddb129f6f1e9/scripts/convert_diffusers_to_original_stable_diffusion.py

In [ ]:
# delete alocated cache
torch.cuda.caching_allocator_delete

<function torch.cuda.memory.caching_allocator_delete(mem_ptr)>

In [ ]:
mount_gdrive = False
drive_path = "/content/mydrive"
model_name = "stabilityai/stable-diffusion-xl-base-1.0"
OUTPUT_DIR = f"sdxl/mymodel-sdxl{model_name.split('-')[-1]}-v1"
if mount_gdrive:
  OUTPUT_DIR = f"{drive_path}/MyDrive/" + OUTPUT_DIR
  drive.mount(drive_path, force_remount=True)

In [ ]:
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path=$model_name \
  --instance_data_dir="/content/data" \
  --output_dir=$OUTPUT_DIR \
  --mixed_precision="fp16" \
  --instance_prompt="your wonderful prompt with a non-existent token, e.g. abrakadababra, for model learning the prompt to image alignment" \
  --seed=1230 \
  --resolution=1024 \
  --gradient_checkpointing \
  --use_8bit_adam \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --checkpointing_steps=500 \
  --max_train_steps=500 \
  --seed="0" \
  --checkpoints_total_limit=5

2023-08-21 10:07:58.635653: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-21 10:08:06.360882: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
08/21/2023 10:08:08 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'variance_type'} was not found in config. Values will be initialized to default values.
{'force_upcast'} was not found in config. Values will be initialized to default values.
{'attention_type'} was not found in config. Values will be initialized to default values.
08/21/

In [ ]:
if wantwant_to_convert_to_chkpt:
    ckpt_path = OUTPUT_DIR + "/model.ckpt"

    half_arg = ""
    fp16 = False
    if fp16:
        half_arg = "--half"
    !python convert_diffusers_to_original_stable_diffusion.py --model_path $OUTPUT_DIR --checkpoint_path $ckpt_path $half_arg
    print(f"[*] Converted ckpt saved at {ckpt_path}")

Traceback (most recent call last):
  File "/content/convert_diffusers_to_original_stable_diffusion.py", line 215, in <module>
    unet_state_dict = torch.load(unet_path, map_location="cpu")
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 791, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 271, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 252, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: '/content/mydrive/MyDrive/sdxl/dog-sdxl0.9-v1/unet/diffusion_pytorch_model.bin'
[*] Converted ckpt saved at /content/mydrive/MyDrive/sdxl/dog-sdxl0.9-v1/model.ckpt


Dummy inference

In [ ]:
import torch
from IPython.display import display
from PIL import Image

from diffusers import StableDiffusionXLImg2ImgPipeline
from diffusers.utils import load_image

In [ ]:
model_name = "stabilityai/stable-diffusion-xl-base-1.0" # repeat it here in case kernel died but the checkpoint is still there
pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_name, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipe = pipe.to("cpu") # otherwise you need more than T4 memory pool

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
pipe.unet.load_attn_procs(f"{OUTPUT_DIR}/checkpoint-10/pytorch_lora_weights.safetensors", use_safetensors=True)
pipe.enable_model_cpu_offload()
init_image = Image.open("path_to_init_image.jpg").convert("RGB")
prompt = "a photo of sakaeatacahaead dog, white background, simple_background"
device="cpu"
generator = torch.Generator(device=device).manual_seed(1024)

In [ ]:
image = pipe(prompt=prompt, 
             image=init_image, 
             strength=0.75, guidance_scale=7.5, 
             generator=generator
             ).images[0]

In [ ]:
image